In [1]:
import SES_setup as ses
import time
import numpy as np
from numpy import pi
from heatmap_setup import calculate_steadystate, heat_map_calculator
from utils import *
reload(ses)
dirname = 'SES_heatmap_excvN'

# Calculating

In [59]:
(5.309e-3)*ev_to_inv_cm

42.819739500000004

In [5]:
reload(ses)
method = 'eigen'
#method = 'power'
#x_values=[6][::-1] # N
x_values=[5,6,7,8,9,10,11,12] # N
#x_values=[3,4,5,6,7][::-1] # N

w_2 = 1.4*ev_to_inv_cm
alpha = 50./pi

pap = alpha_to_pialpha_prop(alpha, w_2)
wc = 100.
w_0 = 200.

Gamma = (w_0**2)/wc
PARAMS = ses.PARAMS_setup(bias=0., w_2=w_2, V = (1e-3)*ev_to_inv_cm, pialpha_prop=pap,
                                T_EM=6000., T_ph =300.,
                                alpha_EM=10, shift=True,
                                num_cpus=4, N=7, Gamma=Gamma, w_0=w_0,
                                silent=True, exc_diff=0)

exc = PARAMS['exc']

 # N
y_values=[5,4,3,2,1,0] # exc
#y_values=[3,2, 1] # exc_diff (exc = 2n-exc_diff)
"""try:
    if 'iterative' in method:
        print("Using fill-factor of {}".format(ff))


ss_array, info_array= heat_map_calculator(PARAMS,
                        x_axis_parameters=['N_1', 'N_2'],
                        y_axis_parameters=['exc_diff'],
                        x_values=x_values,
                        y_values=y_values,
                        dir_name='SES_heatmap_excvN',
                        fill_factor=None, save_data=True, 
                        persistent=False, method=method, threshold=1e-7)
"""
method = 'eigen'
H, L1 = ses.get_H_and_L(PARAMS, silent=False, threshold=1e-7)

The operators took 19.8378338814 and have dimension 147.
((147, 147), (147, 147), (147, 147))
Building the RC Liouvillian took 22.337 seconds.
****************************************************************
It is 21609by21609. The full basis would be 21609by21609
It took  5.2782459259  seconds to build the Non-secular RWA Liouvillian


In [ ]:


SS1, _ = calculate_steadystate(H, L1, persistent=False, method=method)
"""
PARAMS.update({'num_cpus' :1})
H, L2 = ses.get_H_and_L(PARAMS, silent=False, threshold=1e-7)
SS2, _ = calculate_steadystate(H, L2, persistent=False, method=method)

e_op = ses.make_expectation_operators(PARAMS)['XO']
print((SS1*e_op).tr(), (SS2*e_op).tr())
e_op = ses.make_expectation_operators(PARAMS)['site_coherence']
print((SS1*e_op).tr(), (SS2*e_op).tr())

#print np.sum((abs(L1-L2).data))"""


In [27]:
def v0_estimate(H, PARAMS, method=1):
    T1, T2 = PARAMS['T_EM'], 0.5*(PARAMS['T_1']+PARAMS['T_2'])
    alpha1, alpha2 = PARAMS['alpha_EM'], 0.5*(PARAMS['alpha_1']+PARAMS['alpha_2'])
    p1 = alpha1/(alpha1+alpha2)
    p2 = alpha2/(alpha1+alpha2)
    p1/= (p1+p2)
    p2/= (p1+p2)
    if method==1:
        # Average two density matrices together
        r1 = (-beta_f(T1)*H).expm(method='sparse')
        r2 = (-beta_f(T2)*H).expm(method='sparse')
        rho_approx = r1#+r2
        return qt.operator_to_vector(rho_approx/rho_approx.tr()).data.todense()
    elif method==0:
        # Averages temperatures
        r1 = (-beta_f(abs(2*T1-T2))*H).expm(method='sparse')
        return qt.operator_to_vector(r1/r1.tr()).data.todense()
    else:
        r1 = (-beta_f(T1)*H).expm(method='sparse')
        r2 = (-beta_f(T2)*H).expm(method='sparse')
        rho_approx = 0.5*(r1+r2)+(-beta_f(abs(2*T1-T2))*H).expm(method='sparse')
        return qt.operator_to_vector(rho_approx/rho_approx.tr()).data.todense()

In [28]:


for m in [0,1, 2]:
    print "******"
    test = v0_estimate(H[1], PARAMS, method=m)

    #print(qt.operator_to_vector(SS1).data.todense())
    #print(test)
    print sum(abs(qt.operator_to_vector(SS1).data.todense() - test))
    #print H[1].dims
    test_rho = qt.vector_to_operator(qt.Qobj(test, dims=qt.operator_to_vector(SS1).dims))
    print(expectation(test_rho, expval='XO'), expectation(SS1, expval='XO'))
    print(expectation(test_rho, expval='OX'), expectation(SS1, expval='OX'))
    print(expectation(test_rho, expval='OO'), expectation(SS1, expval='OO'))
    print(expectation(test_rho, expval='site_coherence'), expectation(SS1, expval='site_coherence'))

******
[[ 1.65931197]]
((0.16609002692946295+0j), (0.05673757710843215+0j))
((0.16609002692946295+0j), (0.05673757710842914+0j))
(0.6678199461410742, 0.8865248457831385)
((-0.00016472969228066526+0j), (0.001450363320410564+1.408048279209241e-16j))
******
[[ 1.6611555]]
((0.058566694326661436+0j), (0.05673757710843215+0j))
((0.05856669432666145+0j), (0.05673757710842914+0j))
(0.8828666113466771, 0.8865248457831385)
((-0.00011324673684188358+0j), (0.001450363320410564+1.408048279209241e-16j))
******
[[ 1.66288504]]
((0.13722239037972095+0j), (0.05673757710843215+0j))
((0.13722239037972095+0j), (0.05673757710842914+0j))
(0.725555219240558, 0.8865248457831385)
((-0.00014960533986999966+0j), (0.001450363320410564+1.408048279209241e-16j))


In [8]:
v_est = v0_estimate(H[1], PARAMS, method=1)
SS1, _ = calculate_steadystate(H, L1, persistent=False, method='eigen', v0=v_est)

v_est = v0_estimate(H[1], PARAMS, method=0)
SS2, _ = calculate_steadystate(H, L1, persistent=False, method='eigen', v0=v_est)

Steady state took 841.599 seconds
Steady state took 801.846 seconds


In [13]:


print(expectation(SS1))

(0.00145036332041+1.40804827921e-16j)


In [42]:



ev, dm = _steadystate(H, L1, tol=1e-12, sigma=1e-12, ncv=10)
ev, dm = _steadystate(H, L1, tol=1e-12, sigma=1e-12, ncv=15)
ev, dm = _steadystate(H, L1, tol=1e-12, sigma=1e-12, ncv=20)
ev, dm = _steadystate(H, L1, tol=1e-12, sigma=1e-12, ncv=25)
ev, dm = _steadystate(H, L1, tol=1e-12, sigma=1e-12, ncv=30)

Steadystate took 160.341 seconds
[[  5.78691041e-05 +6.14539918e-08j]
 [  7.68968184e-07 -5.24265284e-07j]
 [  3.53352612e-08 -5.74949416e-08j]
 ..., 
 [  2.00874877e-03 -1.46472429e-04j]
 [  1.29804854e-02 -1.03641071e-03j]
 [  5.25408477e-02 +5.57956593e-05j]]
(108, 108) (108, 108)
Coherence is (0.000686369728167+9.02699952548e-17j)
Steadystate took 156.320 seconds
[[  5.65739498e-05 -1.21747769e-05j]
 [  6.40743869e-07 -6.74991445e-07j]
 [  2.23807627e-08 -6.36659285e-08j]
 ..., 
 [  1.93237285e-03 -5.67856448e-04j]
 [  1.24679372e-02 -3.75735149e-03j]
 [  5.13649438e-02 -1.10537930e-02j]]
(108, 108) (108, 108)
Coherence is (0.000686369728167+9.30618158489e-17j)
Steadystate took 136.659 seconds
[[ -6.35751245e-06 -5.75188580e-05j]
 [ -6.06321056e-07 -7.06074312e-07j]
 [ -6.10594201e-08 -2.87401498e-08j]
 ..., 
 [ -3.68370274e-04 -1.98010836e-03j]
 [ -2.46975439e-03 -1.27854394e-02j]
 [ -5.77214903e-03 -5.22228502e-02j]]
(108, 108) (108, 108)
Coherence is (0.000686369728167+1.1859477

# Old Parameters

In [ ]:
reload(ses)
method = 'eigen'
#method = 'power'
#x_values=[6][::-1] # N
x_values=[5,6,7,8,9,10,11,12] # N
#x_values=[3,4,5,6,7][::-1] # N

w_2 = 1.*ev_to_inv_cm
alpha = 50./pi

pap = alpha_to_pialpha_prop(alpha, w_2)
wc = 100.
w_0 = 200.

Gamma = (w_0**2)/wc
PARAMS = ses.PARAMS_setup(bias=100., w_2=w_2, V = 100, pialpha_prop=pap,
                                T_EM=6000., T_ph =300.,
                                alpha_EM=1., shift=True,
                                num_cpus=4, N=7, Gamma=Gamma, w_0=w_0,
                                silent=True, exc_diff=0)

exc = PARAMS['exc']

 # N
y_values=[5,4,3,2,1,0] # exc
#y_values=[3,2, 1] # exc_diff (exc = 2n-exc_diff)
"""try:
    if 'iterative' in method:
        print("Using fill-factor of {}".format(ff))


ss_array, info_array= heat_map_calculator(PARAMS,
                        x_axis_parameters=['N_1', 'N_2'],
                        y_axis_parameters=['exc_diff'],
                        x_values=x_values,
                        y_values=y_values,
                        dir_name='SES_heatmap_excvN',
                        fill_factor=None, save_data=True, 
                        persistent=False, method=method, threshold=1e-7)
"""
method = 'eigen'
H, L1 = ses.get_H_and_L(PARAMS, silent=False, threshold=1e-7)
SS1, _ = calculate_steadystate(H, L1, persistent=False, method=method)
"""PARAMS.update({'num_cpus' :1})
H, L2 = ses.get_H_and_L(PARAMS, silent=False, threshold=1e-7)
SS2, _ = calculate_steadystate(H, L2, persistent=False, method=method)"""

e_op = ses.make_expectation_operators(PARAMS)['XO']
print (SS1*e_op).tr()#, (SS2*e_op).tr())
e_op = ses.make_expectation_operators(PARAMS)['site_coherence']
print (SS1*e_op).tr()#, (SS2*e_op).tr())

#print np.sum((abs(L1-L2).data))

In [ ]:
print(ev_to_inv_cm*1e-3)

In [ ]:
L = qt.liouvillian(H[1], c_ops=[L1])
print (L.data.nnz)

In [ ]:
((SS1*e_op).tr()-(SS2*e_op).tr())

In [ ]:
method = 'eigen'
#method = 'power'
#x_values=[6][::-1] # N
x_values=[5,6,7,8,9,10,11,12] # N
#x_values=[3,4,5,6,7][::-1] # N

w_2 = 8000.
alpha = 50./pi

pap = alpha_to_pialpha_prop(alpha, w_2)
wc = 100.
w_0 = 200.
Gamma = (w_0**2)/wc
PARAMS = ses.PARAMS_setup(bias=100., w_2=8000., V = 100., pialpha_prop=pap,
                                T_EM=6000., T_ph =300.,
                                alpha_EM=0.1, shift=True,
                                num_cpus=12, N=12, Gamma=Gamma, w_0=w_0,
                                silent=True, exc_diff=5)

exc = PARAMS['exc']

 # N
y_values=[5,4,3,2,1,0] # exc
#y_values=[3,2, 1] # exc_diff (exc = 2n-exc_diff)

ss_array, info_array= heat_map_calculator(PARAMS,
                        x_axis_parameters=['N_1', 'N_2'],
                        y_axis_parameters=['exc_diff'],
                        x_values=x_values,
                        y_values=y_values,
                        dir_name='SES_heatmap_excvN',
                        fill_factor=None, save_data=True, 
                        persistent=False, method=method, threshold=1e-7)


# Plotting

In [ ]:
import matplotlib.pyplot as plt

def steadystate_observable(ss_array, PARAMS, observable):
    try:
        size = (len(PARAMS['y_values']), len(PARAMS['x_values']))
    except KeyError:
        size = (1,1)
    # deal with the case where N is being changed as an x value
    if 'N_1' in PARAMS['x_axis_parameters']:
        data_array = np.zeros(size)         # Collect data
        N_values = PARAMS['x_values']
        # NEw observable operator for each N value
        for i, N in enumerate(N_values):
            PARAMS.update({'N_1' : N, 'N_2' : N})
            exc = (2*N)-PARAMS['exc_diff']
            print (N, exc, PARAMS['exc_diff'])
            # Leave it so exc can be fixed or a y_parameter
            if 'exc' in PARAMS['x_axis_parameters']:
                PARAMS.update({'exc': exc})
            _obs_op = make_expectation_operators(PARAMS)[observable]
            for j, y in enumerate(PARAMS['y_values']):
                # the case where exc is on the y axis
                if 'exc_diff' in PARAMS['y_axis_parameters']:
                    PARAMS.update({'exc': (2*N)-y})
                    _obs_op = make_expectation_operators(PARAMS)[observable]
                try:
                    print ss_array[j][i].shape, _obs_op.shape
                    data_array[j][i] = (qt.Qobj(ss_array[j][i])*_obs_op).tr().real
                except AttributeError:
                    data_array[j][i] = np.nan
        return data_array
    else:
        # Much simpler if N isn't on the x-axis
        obs_op = make_expectation_operators(PARAMS)[observable]
        f = lambda x: (qt.Qobj(x)*obs_op).tr()
        ss_array = np.reshape(ss_array, (1, size[0]*size[1]))[0]

        data_array = map(f, ss_array)
        data_array = np.array(data_array).reshape(size).real
        return data_array


def heatmap_plotter(num, plot_SD=True, dir_name="heatmap", 
                     print_params=False, observable='sigma_x', 
                    reverse_x=True):
    
    directory = 'DATA/'+dir_name+'_'+str(num)
    ss_array = load_obj(directory+'/ss_array')
    PARAMS = load_obj(directory+'/PARAMS')
    info_array = load_obj(directory+'/info_array') # reverse ordering -> lower_N-higher_N
    info = info_array[0][0]
    
    try:
        # To catch an old bug where these were in "info" not in PARAMS
        keys = ['x_values', 'y_values', 'y_axis_parameters', 'x_axis_parameters']
        PARAMS.update(dict((key, info[key]) for key in keys))
    except Exception as err:
        print "new style"
    if plot_SD:
        plot_UD_SD(PARAMS["Gamma_1"], PARAMS["alpha_1"], PARAMS["w0_1"], eps=2000.)
    
    ss_array = steadystate_observable(ss_array, PARAMS, observable)

    
    f, axes = plt.subplots(1,1, figsize=(13,6))
    for i, y_val in enumerate(PARAMS['y_values']):
        x_values = PARAMS['x_values']
        if reverse_x:
            x_values = x_values[::-1]
        axes.plot(x_values, ss_array[i], 
                  label=str(PARAMS['y_axis_parameters'])+"="+str(y_val))
    
    print_PARAMS(PARAMS)
    
    plt.xlabel(', '.join(PARAMS['x_axis_parameters']))
    plt.ylabel("Steadystate "+observable)
    #plt.ylabel(', '.join(info['y_axis_parameters']))
    plt.legend()
    plt.grid()
    plot_filename = 'PLOTS/'+dir_name+'_'+str(num)+'.pdf'
    plt.savefig(plot_filename)
    print "saved at {}".format(plot_filename)
    plt.show()

In [ ]:
heatmap_plotter(1, plot_SD=True, dir_name=dirname, 
                     print_params=False, observable='sigma_x', 
                    reverse_x=True)